<a href="https://colab.research.google.com/github/jisazamp/3-column-preview-card-component/blob/main/SistemaRecomendacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score,accuracy_score
import pickle

In [ ]:
datakeys = pd.read_csv("keywords.csv")
datamovies = pd.read_csv("movies_metadata.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'keywords.csv'

In [ ]:
datacredits = pd.read_csv("credits.csv")

In [ ]:
datamovies.info()

In [ ]:
datamovies = datamovies[datamovies["vote_count"] >= 50]

In [ ]:
datamovies = datamovies[["id","genres", "original_title", "overview", "production_companies"]]

In [ ]:
datamovies.head()

In [ ]:
datamovies.describe()

In [ ]:
datamovies.info()

In [ ]:
datamovies.isnull().sum()

In [ ]:
len(datamovies)

In [ ]:
datamovies = datamovies.dropna()

In [ ]:
len(datamovies)

#Concatenar DataSets

In [ ]:
datacredits.head()

In [ ]:
datacredits = datacredits[["id", "cast"]]

In [ ]:
datakeys.head()

In [ ]:
datamovies["id"] = datamovies['id'].astype(int)

In [ ]:
datamovies = pd.merge(datamovies, datakeys, on='id', how='left')

In [ ]:
datamovies.head()

In [ ]:
datamovies.reset_index(inplace=True, drop=True)

In [ ]:
datamovies = pd.merge(datamovies, datacredits, on='id', how='left')

In [ ]:
datamovies.reset_index(inplace=True, drop=True)

In [ ]:
datamovies.head()

In [ ]:
datamovies['genre'] = datamovies['genres'].apply(lambda x: [i['name'] for i in eval(x)])
datamovies.head()

In [ ]:
datamovies['genre'] = datamovies['genre'].apply(lambda x: ' '.join(x))

In [ ]:
datamovies.drop('genres', axis=1, inplace=True)

In [ ]:
datamovies['cast'] = datamovies['cast'].apply(lambda x: [i['name'] for i in eval(x)])
datamovies['cast'] = datamovies['cast'].apply(lambda x: ' '.join(x))

datamovies['production_companies'] = datamovies['production_companies'].apply(lambda x: [i['name'] for i in eval(x)])
datamovies['production_companies'] = datamovies['production_companies'].apply(lambda x: ' '.join(x))

datamovies['keywords'] = datamovies['keywords'].apply(lambda x: [i['name'] for i in eval(x)])
datamovies['keywords'] = datamovies['keywords'].apply(lambda x: ' '.join(x))


datamovies.head()

In [ ]:
datamovies.head()

In [ ]:
datamovies['movie_title'] = datamovies['original_title']

In [ ]:
datamovies.head()

In [ ]:
datamovies['tags'] = datamovies['overview']+' '+datamovies['keywords']+' '+datamovies['cast']+' '+datamovies['genre']+' '+datamovies['original_title'] +' '+datamovies['production_companies']

In [ ]:
datamovies["tags"].head()

In [ ]:
datamovies.drop(['overview', 'keywords', 'cast', 'genre', 'original_title', 'production_companies'], axis=1, inplace = True)

In [ ]:
datamovies.head()

In [ ]:
datamovies.isnull().sum()

Creacion Embeddings


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(max_features=6000)

In [ ]:
vectorized_data = tfidf.fit_transform(datamovies['tags'].values)

In [ ]:
tfidf.get_feature_names_out()

In [ ]:
vectorized_dataframe = pd.DataFrame(vectorized_data.toarray(), index=datamovies['tags'].index.tolist())


In [ ]:
vectorized_dataframe.head()

In [ ]:
vectorized_dataframe.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectorized_dataframe)

In [ ]:
def recommendation_system(movie):
    id_of_movie = datamovies[datamovies['movie_title']==movie].index[0]
    distances = similarity[id_of_movie]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:15]
    for movie_id in movie_list:
        print(datamovies.iloc[movie_id[0]].movie_title)

In [ ]:
recommendation_system("Toy Story")